In [1]:
#交易信息
import pandas as pd
from pathlib import Path
import re
import os
import pandas as pd
import csv
def insert_name(name):
    # 为csv文件添加文件名列
    df = pd.read_csv(name,encoding="ANSI",engine='python',quotechar='"', delimiter='\t",', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    df.columns = df.columns.str.replace('"', '').str.replace('\t', '')
    for col in df.columns:
        if df[col].dtype == 'object':  # Only perform the operation on string columns
            df[col] = df[col].str.replace('"', '').str.replace('\t', '')

    df.insert(0, "文件名", name.stem)


    return df
# 合并csv文件
data=[]
hb=[]

for index,name in enumerate(Path(r"D:\项目\鄂州梁子湖区\返回流水\解压流水").rglob("*交易*.csv")):
    #print(name)
    try:
        df=insert_name(name)
        hb.append(df)
    except:
        print(name)
    
    
data=pd.concat(hb)
#data.columns=data.columns.str.strip()
data['交易卡号']=data['交易卡号'].apply(lambda x:re.split(r'_', x)).apply(lambda x:x[0])
data['交易卡号']=data['交易卡号'].apply(lambda x:re.split(r'-', x)).apply(lambda x:x[0])
data['交易账号']=data['交易账号'].apply(lambda x:re.split(r'_', x)).apply(lambda x:x[0])
data['交易账号']=data['交易账号'].apply(lambda x:re.split(r'-', x)).apply(lambda x:x[0])
data['交易金额'] = pd.to_numeric(data['交易金额'],errors='coerce')
data['交易余额'] = pd.to_numeric(data['交易余额'],errors='coerce')
data=data.drop_duplicates()

In [57]:
data=data.drop(columns=[ '归属目录'], inplace=False)

In [6]:
data.columns

Index(['文件名', '交易卡号', '交易账号', '交易时间', '交易金额', '交易余额', '收付标志', '交易对手账卡号',
       '现金标志', '对手户名', '对手身份证号', '对手开户银行', '摘要说明', '交易币种', '交易网点名称', '交易发生地',
       '交易是否成功', '传票号', 'IP地址', 'MAC地址', '对手交易余额', '交易流水号', '日志号', '凭证种类',
       '凭证号', '交易柜员号', '备注', '查询反馈结果原因', '银行归属'],
      dtype='object')

In [10]:
ls.columns

Index(['交易卡号', '交易账号', '交易时间', '交易金额', '交易余额', '收付标志', '交易对手账卡号', '现金标志',
       '对手户名', '对手身份证号', '对手开户银行', '摘要说明', '交易币种', '交易网点名称', '交易发生地', '交易是否成功',
       '传票号', 'IP地址', 'MAC地址', '对手交易余额', '交易流水号', '日志号', '凭证种类', '凭证号',
       '交易柜员号', '备注', '查询反馈结果原因', '银行归属'],
      dtype='object')

In [2]:
data.shape

(3422136, 28)

In [5]:

# 使用str.extract()函数拆分第一列
data['银行归属'] = data['文件名'].str.extract(r'(.+交易明细信息)')


In [213]:
#data

In [12]:
ls=data.drop(columns=['文件名', '归属目录'], inplace=False)

In [8]:
ls=data.drop(columns=['文件名'], inplace=False)

In [61]:
ls_all.shape

(3422136, 28)

In [9]:
ls.shape

(3422136, 28)

In [127]:
ls.shape

(3032627, 30)

In [11]:
ls=ls.drop_duplicates()

In [118]:
ls_1c922.shape

(1920212, 30)

In [64]:
ls['银行归属'].drop_duplicates()

0    NaN
Name: 银行归属, dtype: object

In [13]:
data_kh_kong=pd.read_excel(r"D:\项目\鄂州梁子湖区\资金分析\账户信息\kong822.xlsx",sheet_name=1,dtype={'交易卡号':str,'交易账号':str})
data_bank=data_kh_kong[['交易卡号','交易账号']]
data_bank=data_bank.drop_duplicates()
bank_dit={k:v for k,v in zip(data_bank['交易账号'],data_bank['交易卡号'])}

In [14]:
def getContinent(x,y):
    if y=='':
        if x in list(filter(lambda zh: zh in bank_dit, bank_dit.keys())):
        #print(x)
        
            val=x.replace(x, bank_dit[x])
        else:
            
            val=y
    
    else:
        val=y
    
    return val

#data['完整卡号'] = data.apply(lambda row: getContinent(row['交易账号'], row['交易卡号']), axis=1)
ls.insert(1,'卡号',ls.apply(lambda row: getContinent(row['交易账号'], row['交易卡号']), axis=1))

In [124]:
#ls_2c_922['流向金额']

In [129]:
ls9477=ls[ls['卡号']=='6228480636323230764']

In [212]:
#ls[ls['卡号']=='6228480636323230764']

In [125]:
ls=ls.drop(columns=[ '流向金额'], inplace=False)

In [16]:
def chuli_chongzheng_lei(i2, f2):
    if pd.isna(i2):
        return f2  # or any appropriate value when i2 is NA

    if i2 in ["出", "D", "借", "转出"]:
        result = -f2
    else:
        result = f2

    return result
ls.insert(4,'流向金额',ls.apply(lambda row: chuli_chongzheng_lei(row['收付标志'], row['交易金额']), axis=1))

In [15]:
need_ka=pd.read_excel(r"D:\项目\鄂州梁子湖区\资金分析\账户信息\账户816发818返反馈.xlsx",sheet_name='会员填单选取收款卡',dtype={'平台收款卡':str})
need_ka.rename(columns = {'平台收款卡':'卡号'},inplace = True)

In [18]:
ls_1c926=ls.merge(need_ka,how='inner',on='卡号')

In [19]:
ls_1c926.shape

(1530704, 30)

In [88]:
ls_1c926.to_csv(r"D:\项目\鄂州梁子湖区\资金分析\ls_1c926.csv",index=False)

In [21]:
ka_skd=pd.read_excel(r"D:\项目\鄂州梁子湖区\资金分析\cs\czjl_t.xlsx",sheet_name='调收款卡',dtype={'NP_PAY_ACCOUNT':str})

In [22]:
import pandas as pd
from datetime import datetime, timedelta
ka_skd.rename(columns = {'NP_PAY_ACCOUNT':'卡号'},inplace = True)
criteria_df=ka_skd
transactions_df=ls_1c926

transactions_df.loc[:, '交易时间'] = pd.to_datetime(transactions_df['交易时间'],errors='coerce')
criteria_df['充值实际最小时间'] = pd.to_datetime(criteria_df['充值实际最小时间'])
criteria_df['充值实际最大时间'] = pd.to_datetime(criteria_df['充值实际最大时间'])

# 定义前后3分钟的时间间隔
delta = timedelta(minutes=4)

# 使用apply函数进行筛选
filtered_df1 = transactions_df.merge(criteria_df, on='卡号', how='left').\
    loc[lambda x: (x['交易时间'] > x['充值实际最小时间'] - delta) & (x['交易时间'] < x['充值实际最大时间'] + delta)]

In [23]:
filtered_df1['流向金额'] = pd.to_numeric(filtered_df1['流向金额'],errors='coerce')
fxb_lx=filtered_df1.groupby(['卡号','交易对手账卡号', '对手户名']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [24]:
fxb_lx_ka=filtered_df1.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [25]:
fxb_lx_wm=filtered_df1.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [26]:
sj_2c_t = filtered_df1.groupby(['交易对手账卡号'])['交易时间'].agg(['min', 'max'])


sj_2c_t = sj_2c_t.rename(columns={'交易对手账卡号':'卡号','min': '最小时间', 'max': '最大时间'})

In [27]:
filtered_df_chu = filtered_df1[filtered_df1['收付标志'].isin(["出", "D", "借", "转出"])]

In [28]:
fxb_chu=filtered_df_chu.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [29]:
fxb_chu_ka=filtered_df_chu.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [30]:
with pd.ExcelWriter(r"D:\项目\鄂州梁子湖区\资金分析\1层分析\1层分析926出.xlsx") as writer:
        fxb_lx.to_excel(writer, sheet_name="fxb_lx",index=False)
        fxb_lx_ka.to_excel(writer, sheet_name="fxb_lx_ka",index=False)
        fxb_lx_wm.to_excel(writer, sheet_name="fxb_lx_wm",index=False)
        sj_2c_t.to_excel(writer, sheet_name="sj_2c_t")
        fxb_chu.to_excel(writer, sheet_name="fxb_chu",index=False)
        fxb_chu_ka.to_excel(writer, sheet_name="fxb_chu_ka",index=False)

In [135]:
#ls9477=ls[ls['卡号']=='6216690100013631626']
ls9477.to_excel(r"D:\项目\鄂州梁子湖区\资金分析\ls0764.xlsx",index=False)

In [31]:
ka_2c=pd.read_excel(r"D:\项目\鄂州梁子湖区\资金分析\1层分析\1层分析922出.xlsx",sheet_name='922二层卡',dtype={'二级卡':str})

In [211]:
#ka_2c

In [199]:
jc=ls[ls['卡号']=='6228480019064062078']

In [200]:
jc.to_excel(r"D:\项目\鄂州梁子湖区\资金分析\cs\fls6228480019064062078.xlsx",index=False)

In [131]:
ka_2c.rename(columns = {'二级卡':'卡号'},inplace = True)

In [33]:
ls_2c_922=ls.merge(ka_2c,how='inner',on='卡号')

In [160]:
sfpf=pd.read_excel(r"D:\项目\鄂州梁子湖区\cs\是否跑分.xlsx",sheet_name='是否跑分',dtype={'卡号':str})

In [161]:
ls_sfpf=ls.merge(sfpf,how='inner',on='卡号')

In [95]:
sfpf=pd.read_excel(r"D:\项目\鄂州梁子湖区\cs\是否跑分.xlsx",sheet_name='是否跑分2',dtype={'卡号':str})

In [187]:
pf=pd.read_excel(r"D:\项目\鄂州梁子湖区\cs\是否跑分.xlsx",sheet_name='跑分卡',dtype={'卡号':str})

In [165]:
#pf

In [184]:
ls_sfpf=ls.merge(pf,how='inner',on='卡号')

In [188]:
ls_pf=ls.merge(pf,how='inner',on='卡号')

In [185]:
ls_sfpf['交易时间'] = pd.to_datetime(ls_sfpf['交易时间'],errors='coerce')

# 提取日期和小时
ls_sfpf['date'] = ls_sfpf['交易时间'].dt.date
ls_sfpf['hour'] = ls_sfpf['交易时间'].dt.hour

# 定义跑分的阈值
count_threshold = 14
count_threshold_night = 10

# 计算每间隔一小时的交易次数，并判断是否在跑分
ls_sfpf_count = ls_sfpf.groupby(['卡号', 'date', 'hour']).size().reset_index(name='count')
ls_sfpf_count['is_running'] = ((ls_sfpf_count['count'] >= count_threshold) |
                          ((ls_sfpf_count['count'] >= count_threshold_night) & (ls_sfpf_count['hour'].between(0, 5))))

# 将符合跑分条件的日期和小时映射到ls_sfpf中
ls_sfpf = ls_sfpf.merge(ls_sfpf_count[['卡号', 'date', 'hour', 'is_running']], on=['卡号', 'date', 'hour'], how='left')

# 填充 'is_running' 列的 NaN 值为 False
ls_sfpf['is_running'] = ls_sfpf['is_running'].fillna(False)

# 计算跑分的总金额
running_sum = ls_sfpf[ls_sfpf['is_running']].groupby('卡号')['交易金额'].sum().reset_index(name='跑分总金额')

running_count = ls_sfpf[ls_sfpf['is_running']].groupby('卡号').size().reset_index(name='跑分总次数')

# 合并跑分总金额和跑分总次数
ls_sfpf_output = pd.merge(running_sum, running_count, on='卡号')
ls_sfpf_output.to_excel(r"D:\项目\鄂州梁子湖区\cs\对手ls_sfpf_output.xlsx",index=False)

In [189]:
ls_pf['交易时间'] = pd.to_datetime(ls_pf['交易时间'],errors='coerce')

# 提取日期和小时
ls_pf['date'] = ls_pf['交易时间'].dt.date
ls_pf['hour'] = ls_pf['交易时间'].dt.hour

# 定义跑分的阈值
count_threshold = 14
count_threshold_night = 10

# 计算每间隔一小时的交易次数，并判断是否在跑分
ls_pf_count = ls_pf.groupby(['卡号', 'date', 'hour']).size().reset_index(name='count')
ls_pf_count['is_running'] = ((ls_pf_count['count'] >= count_threshold) |
                          ((ls_pf_count['count'] >= count_threshold_night) & (ls_pf_count['hour'].between(0, 5))))

# 将符合跑分条件的日期和小时映射到ls_pf中
ls_pf = ls_pf.merge(ls_pf_count[['卡号', 'date', 'hour', 'is_running']], on=['卡号', 'date', 'hour'], how='left')

# 填充 'is_running' 列的 NaN 值为 False
ls_pf['is_running'] = ls_pf['is_running'].fillna(False)


In [190]:
wqpf=ls_pf[ls_pf['is_running']]

In [191]:
pf_t = wqpf.groupby(['卡号'])['交易时间'].agg(['min', 'max'])


pf_t = pf_t.rename(columns={'min': '最小时间', 'max': '最大时间'})

In [192]:
pf_t = pf_t.reset_index()

In [209]:
pfqk=ls_pf[ls_pf['卡号']=='6230520010082806875']

In [210]:
pfqk.to_excel(r"D:\项目\鄂州梁子湖区\资金分析\cs\pfls6230520010082806875.xlsx",index=False)

In [198]:
pfqksx=filtered_pf[filtered_pf['卡号']=='6228480639284773277']
pfqksx.to_excel(r"D:\项目\鄂州梁子湖区\资金分析\cs\sxpfls6228480639284773277.xlsx",index=False)

In [214]:
#pf_t[pf_t['卡号']=='6230520010082806875']

In [193]:
import pandas as pd
from datetime import datetime, timedelta

criteria_df=pf_t
transactions_df=ls_pf
#transactions_df.loc[:,'流向金额']=pd.to_numeric(transactions_df['流向金额'],errors='coerce')
transactions_df.loc[:,'交易时间'] = pd.to_datetime(transactions_df['交易时间'],errors='coerce')
criteria_df['最小时间'] = pd.to_datetime(criteria_df['最小时间'])
criteria_df['最大时间'] = pd.to_datetime(criteria_df['最大时间'])


delta = timedelta(minutes=4)

# 使用apply函数进行筛选
filtered_pf = transactions_df.merge(criteria_df, on='卡号', how='left').\
    loc[lambda x: (x['交易时间'] > x['最小时间'] - delta) & (x['交易时间'] < x['最大时间'] + delta)]

In [123]:
#filtered_pf

In [194]:
#filtered_pf['流向金额'] = pd.to_numeric(filtered_pf['流向金额'],errors='coerce')
fxb_lx=filtered_pf.groupby(['卡号','交易对手账卡号', '对手户名']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_lx_ka=filtered_pf.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_lx_wm=filtered_pf.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
sj_pf_t = filtered_pf.groupby(['交易对手账卡号'])['交易时间'].agg(['min', 'max'])

# 使用rename函数重命名列名
sj_pf_t = sj_pf_t.rename(columns={'交易对手账卡号':'卡号','min': '最小时间', 'max': '最大时间'})
filtered_df_chu = filtered_pf[filtered_pf['收付标志'].isin(["出", "D", "借", "转出"])]
fxb_chu=filtered_df_chu.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_chu_ka=filtered_df_chu.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
with pd.ExcelWriter(r"D:\项目\鄂州梁子湖区\资金分析\跑分1007一二层出.xlsx") as writer:
        fxb_lx.to_excel(writer, sheet_name="fxb_lx",index=False)
        fxb_lx_ka.to_excel(writer, sheet_name="fxb_lx_ka",index=False)
        fxb_lx_wm.to_excel(writer, sheet_name="fxb_lx_wm",index=False)
        sj_3c_t.to_excel(writer, sheet_name="sj_3c_t")
        fxb_chu.to_excel(writer, sheet_name="fxb_chu",index=False)
        fxb_chu_ka.to_excel(writer, sheet_name="fxb_chu_ka",index=False)


In [115]:
#wqpf['流向金额'] = pd.to_numeric(wqpf['流向金额'],errors='coerce')
fxb_lx=wqpf.groupby(['卡号','交易对手账卡号', '对手户名']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_lx_ka=wqpf.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_lx_wm=wqpf.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
sj_pf_t = wqpf.groupby(['交易对手账卡号'])['交易时间'].agg(['min', 'max'])

# 使用rename函数重命名列名
sj_pf_t = sj_pf_t.rename(columns={'交易对手账卡号':'卡号','min': '最小时间', 'max': '最大时间'})
filtered_df_chu = wqpf[wqpf['收付标志'].isin(["出", "D", "借", "转出"])]
fxb_chu=filtered_df_chu.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
fxb_chu_ka=filtered_df_chu.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)
with pd.ExcelWriter(r"D:\项目\鄂州梁子湖区\资金分析\跑分一二层出.xlsx") as writer:
        fxb_lx.to_excel(writer, sheet_name="fxb_lx",index=False)
        fxb_lx_ka.to_excel(writer, sheet_name="fxb_lx_ka",index=False)
        fxb_lx_wm.to_excel(writer, sheet_name="fxb_lx_wm",index=False)
        sj_3c_t.to_excel(writer, sheet_name="sj_3c_t")
        fxb_chu.to_excel(writer, sheet_name="fxb_chu",index=False)
        fxb_chu_ka.to_excel(writer, sheet_name="fxb_chu_ka",index=False)

In [71]:
import pandas as pd
from datetime import datetime, timedelta
ka_2c_t=pd.read_excel(r"D:\项目\鄂州梁子湖区\资金分析\1层分析\1层分析922出.xlsx",sheet_name='922二层卡时间范围',dtype={'二级卡':str})
ka_2c_t.rename(columns = {'二级卡':'卡号'},inplace = True)
criteria_df=ka_2c_t
transactions_df=ls_2c_922
#transactions_df.loc[:,'流向金额']=pd.to_numeric(transactions_df['流向金额'],errors='coerce')
transactions_df.loc[:,'交易时间'] = pd.to_datetime(transactions_df['交易时间'],errors='coerce')
criteria_df['最小时间'] = pd.to_datetime(criteria_df['最小时间'])
criteria_df['最大时间'] = pd.to_datetime(criteria_df['最大时间'])


delta = timedelta(minutes=4)

# 使用apply函数进行筛选
filtered_df2 = transactions_df.merge(criteria_df, on='卡号', how='left').\
    loc[lambda x: (x['交易时间'] > x['最小时间'] - delta) & (x['交易时间'] < x['最大时间'] + delta)]

In [72]:
ls9477=filtered_df2[filtered_df2['卡号']=='6228480636323230764']

In [215]:
#ls9477

In [216]:
#ls9477[ls9477['对手户名']=='刘航']

In [48]:
ls9477.to_csv(r"D:\项目\鄂州梁子湖区\资金分析\cs\fls0764.csv",index=False)

In [51]:
jc=ls_2c_922[ls_2c_922['卡号']=='6228480636323230764']

In [60]:
#jc[jc['对手户名']=='刘航']

In [35]:
#ls9477.iloc[1280:1285,:]

In [75]:
#filtered_df2['流向金额'] = pd.to_numeric(filtered_df2['流向金额'],errors='coerce')
fxb_lx=filtered_df2.groupby(['卡号','交易对手账卡号', '对手户名']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [76]:
fxb_lx_ka=filtered_df2.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [77]:
fxb_lx_wm=filtered_df2.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [78]:
sj_3c_t = filtered_df2.groupby(['交易对手账卡号'])['交易时间'].agg(['min', 'max'])

# 使用rename函数重命名列名
sj_3c_t = sj_3c_t.rename(columns={'交易对手账卡号':'卡号','min': '最小时间', 'max': '最大时间'})

In [79]:
filtered_df_chu = filtered_df2[filtered_df2['收付标志'].isin(["出", "D", "借", "转出"])]

In [80]:
fxb_chu=filtered_df_chu.groupby(['交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [81]:
fxb_chu_ka=filtered_df_chu.groupby(['卡号','交易对手账卡号']).agg(sum=('流向金额', 'sum'), count=('流向金额', 'count')).reset_index().sort_values('sum', ascending=True)

In [82]:
with pd.ExcelWriter(r"D:\项目\鄂州梁子湖区\资金分析\二层分析\2层分析926出.xlsx") as writer:
        fxb_lx.to_excel(writer, sheet_name="fxb_lx",index=False)
        fxb_lx_ka.to_excel(writer, sheet_name="fxb_lx_ka",index=False)
        fxb_lx_wm.to_excel(writer, sheet_name="fxb_lx_wm",index=False)
        sj_3c_t.to_excel(writer, sheet_name="sj_3c_t")
        fxb_chu.to_excel(writer, sheet_name="fxb_chu",index=False)
        fxb_chu_ka.to_excel(writer, sheet_name="fxb_chu_ka",index=False)